# Baixa e limpa os dados

In [13]:
# Geospatial data processing
import geopandas as gpd

# Import a biblioteca de barra de progresso
from tqdm.auto import tqdm 

# Mapping and visualization
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# The star of the show: city2graph for transportation network analysis
import city2graph as c2g
import city2graph.data as c2g_data
import city2graph.utils as c2g_utils

# Configure matplotlib for publication-quality visualizations
plt.rcParams['figure.figsize'] = (15, 12)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = 10
plt.style.use('ggplot')
#plt.style.use('default')  # Clean default style instead of ggplot

print("All dependencies loaded successfully!")
print(f"city2graph version: {c2g.__version__ if hasattr(c2g, '__version__') else 'development'}")

All dependencies loaded successfully!
city2graph version: 0.2.3


---

#### Só execute a célula abaixo, caso já tenha feito download dos dados brutos e quer fazer alguma checagem nesses dados originais

---

In [ ]:
gdf_segments = gpd.read_file(f'{path_brutos}/the_bbox_segment.geojson')
gdf_connector = gpd.read_file(f'{path_brutos}/the_bbox_connector.geojson')

---
### Fazendo download dos geodados de rede viária (segmentos e conectores) da OvertureMaps em formato GeoJSON.
---

In [14]:
# Caixa delimitadora (Bounding Box, BBOX) do municípios
# [min_lon, min_lat, max_lon, max_lat]
bbox3km_zonaUrbana = [-42.863200,-5.259269, -42.626205, -4.929381] # BBOX do buffer de 3km da zona urbana
bbox_teresina = [-42.98,-5.59,-42.60,-4.79] # BBOX da município

path_brutos = "../dados/brutos/overturemaps/"
types = ["segment", "connector"]

In [15]:
overture_data = c2g_data.load_overture_data(
    area=bbox_teresina, # Área da zona leste de Teresina
    types=types,
    output_dir=path_brutos,
    prefix="the_bbox_",
    save_to_file=True,
    return_data=True
)

print(f"O download dos dados {types} foi concluído.")

O download dos dados ['segment', 'connector'] foi concluído.


In [16]:
gdf_segmentos = overture_data['segment']
gdf_conectores = overture_data['connector']

In [20]:
print(f'\nSegmentos: {gdf_segmentos.shape[0]} linhas e {gdf_segmentos.shape[1]} colunas')


Segmentos: 66486 linhas e 20 colunas


In [21]:
print(f'\nConectores {gdf_conectores.shape[0]} linhas e {gdf_conectores.shape[1]} colunas')


Conectores 49664 linhas e 4 colunas


In [23]:
gdf_segmentos.head(2)

,id,version,sources,subtype,class,names,connectors,routes,subclass_rules,access_restrictions,level_rules,destinations,prohibited_transitions,rail_flags,road_surface,road_flags,speed_limits,width_rules,subclass,geometry
0,6d775c41-5cf2-493e-b402-cfcbd8972262,1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",road,unclassified,NaN,[{'connector_id': 'eaaeee83-1ec8-4c2f-a46e-dde...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'unpaved', 'between': None}]",NaN,NaN,NaN,NaN,"LINESTRING (-42.98426 -5.34934, -42.98401 -5.3..."
1,aec4047f-e9b3-4c2b-ba48-c1c065ffc97e,1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",road,unclassified,NaN,[{'connector_id': '89a3c9a6-e65e-4102-888f-499...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'unpaved', 'between': None}]",NaN,NaN,NaN,NaN,"LINESTRING (-42.97465 -5.28342, -42.97852 -5.2..."


In [22]:
gdf_conectores.head(3)

,id,version,sources,geometry
0,1a9f6ba4-85ea-4623-b796-403dac35c312,1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",POINT (-42.97852 -5.34666)
1,c32213b5-fe58-4ca9-a910-32aec6ff0181,1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",POINT (-42.97931 -5.29226)
2,3d80bce1-d842-459a-9195-f849da94e6fe,1,"[{'property': '', 'dataset': 'OpenStreetMap', ...",POINT (-42.95956 -5.30976)


In [ ]:
# -------------------------------------- #
# TIPOS DE VIA
# -------------------------------------- #
print(f'\n foram encontrados {len(gdf_segments['subtype'].unique().tolist())} tipos de segmentos:\n{gdf_segments['subtype'].unique().tolist()}')


 foram encontrados 2 tipos de segmentos:
['road', 'rail']


In [29]:
# --------------------------------------
# CLASSE DAS VIA
# --------------------------------------
print(f'\n foram encontrados {len(gdf_segments['class'].unique().tolist())} classe de segmentos:\n{gdf_segments['class'].unique().tolist()}')


 foram encontrados 17 classe de segmentos:
['path', 'unclassified', 'tertiary', 'track', 'primary', 'residential', 'trunk', 'service', 'footway', 'unknown', 'living_street', 'secondary', 'pedestrian', 'cycleway', 'steps', 'standard_gauge', 'light_rail']


In [30]:
# --------------------------------------
# SUBCLASSE DAS VIA
# --------------------------------------
print(f'\n foram encontrados {len(gdf_segments['subclass'].unique().tolist())} classe de segmentos:\n{gdf_segments['subclass'].unique().tolist()}')


 foram encontrados 8 classe de segmentos:
[nan, 'link', 'crosswalk', 'driveway', 'alley', 'sidewalk', 'parking_aisle', 'cycle_crossing']


In [33]:
gdf_segments.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [34]:
# Garantir projeção métrica (fundamental para Alpha Shapes funcionarem bem)
if gdf_segments.crs.to_string() != "EPSG:31983":
    print(f"Segmentos -> CRS Fonte = {gdf_segments.crs.to_string()}. Reprojetando segmentos para SIRGAS2000/UTM23Sul (EPSG:31983)...")
    gdf_segments_utm = gdf_segments.to_crs(epsg=31983)

if gdf_conectores.crs.to_string() != "EPSG:31983":
    print(f"Conectores -> CRS Fonte = {gdf_conectores.crs.to_string()}. Reprojetando segmentos para SIRGAS2000/UTM23Sul (EPSG:31983)...")
    gdf_conectores_utm = gdf_conectores.to_crs(epsg=31983)

Segmentos -> CRS Fonte = EPSG:4326. Reprojetando segmentos para SIRGAS2000/UTM23Sul (EPSG:31983)...
Conectores -> CRS Fonte = EPSG:4326. Reprojetando segmentos para SIRGAS2000/UTM23Sul (EPSG:31983)...
